# AEDT Field Export Notebook

This notebook is for generating *new* lead field archives (`.npz` / optional `.mat`) from ANSYS AEDT simulations. It:
- Iterates electrodes (channels) and exports individual field gain matrices.
- Reassembles per-electrode `.lead` files into a unified 5‑D tensor.
- Saves compressed archives for downstream optimization scripts (`leadfield_importer`).

Prerequisites
1. A licensed local ANSYS AEDT install with solved electrostatic or bioelectric model.
2. Simulated field solution per electrode (each channel excited individually).
3. A writable output directory (created beforehand) for temporary `.lead` files.

Output Structure
```
(X, Y, Z, 3, E)
X,Y,Z : spatial grid indices
3     : field components (Ex,Ey,Ez) or potential-derived vector
E     : electrode index
```

Good Practices
- Keep simulation & export grid parameters consistent (avoid mixing voxel sizes).
- Document electrode pitch, conductivity assumptions, and any symmetries.
- Use version control for exported archives (`SEEG_8e_500um_1500pitch.npz`).
- Record simulation metadata (date, AEDT version) for reproducibility.


## Settings
Adjust the variables below to point to your dataset and AEDT project.

Required:
- `folder`: Root of unzipped SEPIO dataset containing `ansys/` subfolder.
- `file_name`: Base name of simulation (used for folder and archive filenames).
- `num_electrodes`: Must match number of individually excited electrodes in AEDT.

Paths Created/Used:
- `leadfield_dir`: Directory where raw `.lead` files will be written.
- `ansys_project_path`: Path to the AEDT project file (`.aedt`).
- `archive_file_np` / `archive_file_mat`: Output archive destinations.

Notes:
- Create `leadfield_dir` BEFORE running export. (Notebook will not auto-create.)
- Set `export_numpy` or `export_mat` depending on downstream tooling (MATLAB compatibility).


In [ ]:
# Local imports
from os import path
from modules.leadfield_exporter import LeadFieldExporter

# File management & numerics
from os import path, curdir
import os
import numpy as np

# ------------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------------
# Root dataset directory (UNZIPPED). Replace placeholder with real path.
folder = r"...\SEPIO_dataset"  # e.g., r"C:\Data\SEPIO_dataset"

# Base simulation name (used for directory & archives)
file_name = 'SEEG_8e_500um_1500pitch'  # Examples: 'IMEC', 'SEEG', 'DISC', 'ECOG'

# Electrode count in solved AEDT model; MUST match exported channels
num_electrodes = 8

# Derived paths
leadfield_dir = path.join(folder, 'ansys', file_name)
ansys_project_path = path.join(folder, 'ansys', file_name + '.aedt')
archive_file_np = path.join(folder, 'ansys', file_name + '.npz')
archive_file_mat = path.join(folder, 'ansys', file_name + '.mat')

# Export format toggles
export_numpy = True
export_mat = False  # Enable if MATLAB consumption required

# ------------------------------------------------------------------
# VALIDATION (non-fatal warnings to guide user)
# ------------------------------------------------------------------
if not os.path.isdir(folder):
    print(f"[WARN] Root folder does not exist: {folder}")
if not os.path.isfile(ansys_project_path):
    print(f"[WARN] AEDT project not found: {ansys_project_path}")
if not os.path.isdir(leadfield_dir):
    print(f"[WARN] Leadfield directory missing (create before export): {leadfield_dir}")
if num_electrodes <= 0:
    raise ValueError("num_electrodes must be > 0")

print("Configured leadfield export:")
print(f"  Project: {ansys_project_path}")
print(f"  Output dir: {leadfield_dir}")
print(f"  Electrodes: {num_electrodes}")
print(f"  Archives: NPZ={export_numpy}, MAT={export_mat}")


## Export Lead Field Data
Run the cell below to:
1. Export per-electrode field data across a defined spatial grid.
2. Import all generated `.lead` files.
3. Assemble into the unified tensor.
4. Save archives (numpy and/or MATLAB).

Safety Checks:
- Ensure grid extents (`grid_start`, `grid_stop`) fully cover simulated region.
- Choose a voxel step that balances resolution vs memory/time.
- Do NOT overwrite existing validated archives without versioning.


## Grid Parameter Examples
Select grid parameters aligned with the tissue volume solved in AEDT.

Recommended Sets:

Default 10 mm Region:
- `grid_start = [-5, -5, 0]`
- `grid_stop  = [5, 5, 10]`
- `grid_step  = [0.1, 0.1, 0.1]`

Full Brain (Higher Coverage):
- `grid_start = [-8.5, -11.5, 0]`
- `grid_stop  = [8.5, 11.5, 11]`
- `grid_step  = [0.1, 0.1, 0.1]`

30 mm Device-Focused:
- `grid_start = [-15, -15, 0]`
- `grid_stop  = [15, 15, 10]`
- `grid_step  = [0.5, 0.5, 0.5]`

Guidance:
- Smaller `grid_step` increases resolution and file size/time.
- Ensure Z-range matches electrode vertical extent in simulation.
- Keep units consistent (millimeters) across all settings and AEDT.


In [ ]:
# Note: MUST match the region of simulated tissue in ANSYS model
# Adjust these before running; see example sets above.
grid_start = [-15, -15, 0]   # X, Y, Z start (mm)
grid_stop  = [15, 15, 10]     # X, Y, Z stop (mm)
grid_step  = [0.5, 0.5, 0.5]  # voxel size (mm)

# Optional: set electrode positions if available (used for .mat export metadata)
elec_pos = False  # or np.array([...])

try:
    exporter = LeadFieldExporter(ansys_project_path)
    exporter.export_fields_on_grid(
        channels=range(1, num_electrodes + 1),
        grid_start=grid_start,
        grid_stop=grid_stop,
        grid_step=grid_step,
        output_dir=leadfield_dir
    )
finally:
    exporter.import_fields(leadfield_dir)
    if elec_pos is not False:
        print("Attaching electrode position metadata.")
        exporter.metadata = {
            'electrode_positions': elec_pos,
            'electrode_pitch': 0.5,  # mm (example; adjust)
            'conductivity': 0.2      # S/m (example; document actual value)
        }
    if export_numpy:
        exporter.save(archive_file_np, type='numpy')
        print(f"Saved numpy archive: {archive_file_np}")
    if export_mat:
        exporter.save(archive_file_mat, type='matlab')
        print(f"Saved MATLAB archive: {archive_file_mat}")
    exporter.close()


## MATLAB Export & Electrode Metadata
If exporting to MATLAB (`export_mat = True`), you may include electrode position metadata. This enables downstream spatial analyses without re-parsing geometry.

Provide:
- `electrode_positions`: array of shape `(E, 3)` in millimeters.
- `electrode_pitch`: scalar distance between adjacent contacts (if uniform).
- `conductivity`: assumed bulk tissue conductivity used in simulation (S/m).

Leave positions unset (`False`) if not available; importer defaults apply.


In [ ]:
# Electrode position example (unused by default)
# This large array demonstrates a possible mixed linear & radial/contact mapping.
# For real use, trim to actual electrode count or generate programmatically.
# Assign to `elec_pos` above before export if needed.

elec_pos = np.array([
    [0.405, 0, 1.75],[0.405, 0, 2.25],[0.405, 0, 2.75],[0.405, 0, 3.25],[0.405, 0, 3.75],[0.405, 0, 4.25],[0.405, 0, 4.75],[0.405, 0, 5.25],
    [0.405, 0, 5.75],[0.405, 0, 6.25],[0.405, 0, 6.75],[0.405, 0, 7.25],[0.405, 0, 7.75],[0.405, 0, 8.25],[0.405, 0, 8.75],[0.405, 0, 9.25],
    [0.2863782464, 0.2863782464, 2],[0.2863782464, 0.2863782464, 2.5],[0.2863782464, 0.2863782464, 3],[0.2863782464, 0.2863782464, 3.5],
    [0.2863782464, 0.2863782464, 4],[0.2863782464, 0.2863782464, 4.5],[0.2863782464, 0.2863782464, 5],[0.2863782464, 0.2863782464, 5.5],
    [0.2863782464, 0.2863782464, 6],[0.2863782464, 0.2863782464, 6.5],[0.2863782464, 0.2863782464, 7],[0.2863782464, 0.2863782464, 7.5],
    [0.2863782464, 0.2863782464, 8],[0.2863782464, 0.2863782464, 8.5],[0.2863782464, 0.2863782464, 9],[0.2863782464, 0.2863782464, 9.5],
    [0, 0.405, 1.75],[0, 0.405, 2.25],[0, 0.405, 2.75],[0, 0.405, 3.25],[0, 0.405, 3.75],[0, 0.405, 4.25],[0, 0.405, 4.75],[0, 0.405, 5.25],
    [0, 0.405, 5.75],[0, 0.405, 6.25],[0, 0.405, 6.75],[0, 0.405, 7.25],[0, 0.405, 7.75],[0, 0.405, 8.25],[0, 0.405, 8.75],[0, 0.405, 9.25],
    [-0.2863782464, 0.2863782464, 2],[-0.2863782464, 0.2863782464, 2.5],[-0.2863782464, 0.2863782464, 3],[-0.2863782464, 0.2863782464, 3.5],
    [-0.2863782464, 0.2863782464, 4],[-0.2863782464, 0.2863782464, 4.5],[-0.2863782464, 0.2863782464, 5],[-0.2863782464, 0.2863782464, 5.5],
    [-0.2863782464, 0.2863782464, 6],[-0.2863782464, 0.2863782464, 6.5],[-0.2863782464, 0.2863782464, 7],[-0.2863782464, 0.2863782464, 7.5],
    [-0.2863782464, 0.2863782464, 8],[-0.2863782464, 0.2863782464, 8.5],[-0.2863782464, 0.2863782464, 9],[-0.2863782464, 0.2863782464, 9.5],
    [-0.405, 0, 1.75],[-0.405, 0, 2.25],[-0.405, 0, 2.75],[-0.405, 0, 3.25],[-0.405, 0, 3.75],[-0.405, 0, 4.25],[-0.405, 0, 4.75],[-0.405, 0, 5.25],
    [-0.405, 0, 5.75],[-0.405, 0, 6.25],[-0.405, 0, 6.75],[-0.405, 0, 7.25],[-0.405, 0, 7.75],[-0.405, 0, 8.25],[-0.405, 0, 8.75],[-0.405, 0, 9.25],
    [-0.2863782464, -0.2863782464, 2],[-0.2863782464, -0.2863782464, 2.5],[-0.2863782464, -0.2863782464, 3],[-0.2863782464, -0.2863782464, 3.5],
    [-0.2863782464, -0.2863782464, 4],[-0.2863782464, -0.2863782464, 4.5],[-0.2863782464, -0.2863782464, 5],[-0.2863782464, -0.2863782464, 5.5],
    [-0.2863782464, -0.2863782464, 6],[-0.2863782464, -0.2863782464, 6.5],[-0.2863782464, -0.2863782464, 7],[-0.2863782464, -0.2863782464, 7.5],
    [-0.2863782464, -0.2863782464, 8],[-0.2863782464, -0.2863782464, 8.5],[-0.2863782464, -0.2863782464, 9],[-0.2863782464, -0.2863782464, 9.5],
    [0, -0.405, 1.75],[0, -0.405, 2.25],[0, -0.405, 2.75],[0, -0.405, 3.25],[0, -0.405, 3.75],[0, -0.405, 4.25],[0, -0.405, 4.75],[0, -0.405, 5.25],
    [0, -0.405, 5.75],[0, -0.405, 6.25],[0, -0.405, 6.75],[0, -0.405, 7.25],[0, -0.405, 7.75],[0, -0.405, 8.25],[0, -0.405, 8.75],[0, -0.405, 9.25],
    [0.2863782464, -0.2863782464, 2],[0.2863782464, -0.2863782464, 2.5],[0.2863782464, -0.2863782464, 3],[0.2863782464, -0.2863782464, 3.5],
    [0.2863782464, -0.2863782464, 4],[0.2863782464, -0.2863782464, 4.5],[0.2863782464, -0.2863782464, 5],[0.2863782464, -0.2863782464, 5.5],
    [0.2863782464, -0.2863782464, 6],[0.2863782464, -0.2863782464, 6.5],[0.2863782464, -0.2863782464, 7],[0.2863782464, -0.2863782464, 7.5],
    [0.2863782464, -0.2863782464, 8],[0.2863782464, -0.2863782464, 8.5],[0.2863782464, -0.2863782464, 9],[0.2863782464, -0.2863782464, 9.5]
])

print(f"Example electrode positions loaded: shape={elec_pos.shape}")
